## 통합 데이터 지도를 통한 배달과 날씨 관련 데이터 찾기
* 통합 데이터 지도에서는 16대 빅데이터 플랫폼과 AI Hub, 데이터스토어, Kotra, 한국관광공사, 쿠콘 등 데이터를 찾아볼 수 있습니다.
* 공공데이터 뿐만 아니라 민간데이터 현황도 한 눈에 볼 수 있습니다.
* 여기에서는 통합데이터지도에서 제공하고 있는 민간 데이터 중 하나인 KT 통신 빅데이터 플랫폼에서 제공하고 있는 `기상정보를 활용한 배달 품목의 차이` 데이터를 분석해 보도록 하겠습니다.
* 민간 데이터 플랫폼에서는 유료 데이터도 제공하고 있으나 비용없이 민간 데이터를 사용해 보기 위해 무료 데이터로 제공하고 있는 `기상정보를 활용한 배달 품목의 차이` 를 보겠습니다.
* 해당 데이터의 위치는 아래 URL 에서 찾아볼 수 있으며, KT 통신 빅데이터 플랫폼에서 다운로드가 가능합니다.
* 해당 데이터가 무료 데이터일지라도 다운로드를 위해서는 해당 플랫폼에 회원가입을 하고 로그인을 해야 다운로드가 가능합니다.

* [통합 데이터지도에서 날씨와 배달로 검색한 결과](https://www.bigdata-map.kr/search/theme?searchKey=%EB%82%A0%EC%94%A8%20%EB%B0%B0%EB%8B%AC)
* [통합 데이터지도 - 기상정보를 활용한 배달 품목의 차이](https://www.bigdata-map.kr/search/1311919)
* [KT 통신 빅데이터 플랫폼 데이터마켓](https://www.bigdata-telecom.kr/invoke/SOKBP2603/?goodsCode=KGUWTHRDLVRDF)


## 기상정보를 활용한 배달 품목의 차이

* KT 통신 빅데이터 플랫폼 데이터마켓에서 일부 데이터를 미리보기 할 수 있습니다. CSV 파일과 엑셀파일로 제공하고 있으며, CSV파일은 3개, 데이터에 대한 설명이 있는 엑셀파일이 있습니다.
* 총 4개의 파일을 사용해서 표 형태의 하나의 데이터프레임으로 만들고 분석을 해볼 예정입니다.
* 날짜별로 기상정보와 배달음식 품목별 수량이 나와있습니다. 
* 비가 오는 날에는 기름진 음식이 먹고 싶고, 평소 한식을 많이 먹을 것이라는 가설을 세워보겠습니다.
* 그럼 데이터를 통해 가설이 얼마나 맞는지 분석해 보겠습니다.

## 라이브러리 로드
* 분석에 필요한 라이브러리를 로드합니다. 
* glob 는 파일 목록을 유닉스 스타일의 규칙에 따라 읽어오게 됩니다.
* pandas 는 파이썬의 대표적인 데이터 분석 도구 입니다.
* seaborn 은 통계적 연산을 제공하고 matplotlib 을 사용하기 쉽게 감싸놓은 시각화 도구입니다.
* plotly.express 는 동적인 시각화를 할 수 있는 plotly 의 추상화된 기능을 제공하여 쉬운 인터페이스를 통해 시각화가 가능합니다.
* matplotlib.pyplot 은 파이썬의 대표적인 정적인 시각화 도구입니다.

In [ ]:
from glob import glob
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import koreanize_matplotlib
plt.style.use("ggplot")

* glob 를 통해 파일을 읽어옵니다. KGU*.csv 는 KGU로 시작하고 .csv 로 끝나는 파일을 찾아오도록 합니다.

In [ ]:
file_names = glob("data/KGU*.csv")
file_names

* 읽어올 파일이 여러 개라면 데이터프레임도 그 수만큼 불러오게 됩니다. 이 때 하나의 데이터프레임으로 되어 있어야 분석이 편합니다.
* 리스트 컴프리헨션 기능을 사용하여 읽어오고 pandas의 concat 으로 합쳐주게 되면 하나의 데이터프레임으로 합쳐볼 수 있습니다.

In [ ]:
df_file = [pd.read_csv(fname, header=None) for fname in file_names]
df = pd.concat(df_file)
df.shape

* 데이터를 불러왔다면 head(), tail(), sample() 등으로 일부 데이터만 가져와서 미리보기를 합니다.
* 컬럼값이 숫자로 되어 있어서 전처리가 필요해 보입니다.

In [ ]:
df.head()

* KGUWTHRDLVRDF_DataDescription 파일을 열어보면 데이터에 대한 설명이 있는 엑셀 파일이 있습니다.
* 해당 엑셀파일을 열어보면 컬럼에 대한 한글, 영문명이 있으며 데이터에 대한 설명도 볼 수 있습니다.

In [ ]:
df_cols = pd.read_excel("data/KGUWTHRDLVRDF_DataDescription.xlsx")
df_cols

* 컬럼을 보기 좋게 한글로 변경하기 위해 기존 DataDescription 엑셀 파일에서 한글로 된 컬럼명만 가져옵니다.
* pandas 의 컬럼인덱싱과 .values Attributes를 사용합니다.
* 이 때, cols 라는 변수에 할당해서 재사용 합니다.

In [ ]:
cols = df_cols["컬럼한글명"].values
cols

* df.columns 에 위에서 구한 한글컬럼이 들어있는 cols를 지정해 주면 숫자로 되어있던 컬럼명이 직관적으로 변경됩니다.

In [ ]:
df.columns = cols
df

## 요약과 기술통계
* info 를 통해 전체적으로 어떤 데이터가 있는지 확인합니다.
* 해당 데이터는 날씨별 배달품목의 수를 확인할 수 있습니다.
* 날씨별로 어떤 메뉴를 더 많이 주문할지 분석해 볼 수 있습니다.
* 음식 관련 서비스나 배달 서비스를 운영하고 있는 회사라면 이 데이터를 통해 날씨별로 어떤 메뉴를 더 프로모션하거나 재료를 준비해야 할지에 대한 참고자료로 활용이 가능할 것입니다.

In [ ]:
# cols 에는 모든 컬럼명이 있습니다. 여기에서 _배달건수 가 들어가는 컬럼은 음식 종류이기 때문에 어떤 음식종류가 있는지만 따로 정리합니다.
cols

In [ ]:
# 리스트 컴프리헨션을 사용하여 분석해야 할 배달음식 종류가 무엇인지를 확인합니다.
[c.split("_")[0] for c in cols if "배달" in c]

In [ ]:
# info 는 전체적인 데이터프레임의 정보를 보여줍니다. 
# 행과 열의 수, 메모리 사용량, 컬럼명, 결측치 수, 데이터 타입 등을 확인해 볼 수 있습니다.
df.info()

In [ ]:
# 결측치가 있는지 확인합니다. isnull()은 결측치 여부를 True, False 로 반환하며, 
# 해당 값의 합계를 sum()을 사용해서 구하면 결측치의 합계가 됩니다.
df.isnull().sum()

* describe()는 기술통계를 구할 수 있습니다. 기술통계는 숫자와 문자데이터에 대한 요약통계를 보여줍니다.
* describe() 는 별도의 옵션을 지정하지 않았을 때 수치 데이터에 대한 기술 통계를 보여줍니다.
* count 는 결측치를 제외한 빈도수, mean 은 평균, std 는 표준편차, min은 최솟값, 25% 는 데이터를 한 줄로 세웠을 때 25% 위치에 있는 값으로 4등분 했다하여 사분위수라고도 하며, 25% 는 1사분위수가 됩니다. 50%는 2사분위수로 중앙값을 나타냅니다. 75%는 3사분위수 입니다. max 는 최댓값 입니다.

In [ ]:
df.describe()

* describe() 에 include, exclude 등의 옵션을 지정할 수 있으며 include="object" 는 문자 데이터에 대한 기술통계값을 보여줍니다.
* count는 결측치를 제외한 빈도수, unique는 중복을 제외한 유일값의 개수, top 은 최빈값, freq 는 최빈값에 대한 빈도를 나타냅니다.

In [ ]:
df.describe(include="object")

* 기술통계의 4분위수를 보여주기 가장 좋은 그래프는 boxplot입니다. 
* seaborn의 boxplot 으로 시각화를 해보면 성격이 다른 변수가 섞여있어 박스의 범위가 제대로 보이지 않습니다. 
* 또, 이 때 orient="h" 를 사용했는데 h는 그래프를 가로로 그리고 v는 세로로 그리게 됩니다.
* 참고로 v와 h는 vertical과 horizontal의 약자입니다. 위 설명과 마찬가지로 세로, 가로를 의미합니다.
* 여기에서 orient="h" 를 사용한 이유는 각 변수의 값을 y축에 표현하면 텍스트가 겹쳐보이지 않고 보기에 좋기 때문입니다.

In [ ]:
sns.boxplot(data=df, orient="h")

* 다른 변수가 함께 있어서 보기에 어렵습니다. 배달건수가 들어가는 컬럼만 가져와서 분석해 봅니다.

In [ ]:
df.filter(like="배달건수")

In [ ]:
# filter는 특정 패턴으로 인덱스와 컬럼의 값을 가져올 수 있습니다.
# 여기에서는 "배달건수" 라는 컬럼이 들어간 데이터프레임만을 구해 boxplot을 시각화 했습니다.
sns.boxplot(df.filter(like="배달건수"), orient="h")

In [ ]:
# 값의 분포를 확인했다면 대표값을 시각화 해봅니다.
# seaborn 의 barplot은 연산을 위한 estimator를 지정하지 않으면 평균을 구합니다.
# 치킨이 평균적으로 배달건수가 많음을 확인해 볼 수 있습니다.
sns.barplot(df.filter(like="배달건수"), orient="h", errorbar=None).set_title("날짜별 시간대별 평균 음식 배달 건수")

In [ ]:
# 이 데이터는 날짜별로 강수 여부가 있습니다. 강수 유형별 빈도수를 보면 강수가 없는 날이 대부분입니다.
df["강수 유형명"].value_counts()

In [ ]:
# 강수 유형별로 치킨 배달건수를 구해봅니다. 
# 이 때, 합계를 구하게 되면 비가 오지 않는 날이 대부분이기 때문에 강수가 없는 날이 빈도가 높을 수 밖에 없습니다.
df.groupby("강수 유형명")["치킨_배달건수"].sum().plot.bar(rot=0)

In [ ]:
# 강수 유형별로 배달건수의 평균을 구하면 진눈개비가 내리는 날 치킨 배달건수가 평균적으로 더 많다는 것을 확인할 수 있습니다.
# 음식 종류가 컬럼으로 되어 있기 때문에 각 음식별로 배달건수를 구해야 하는데 음식 종류가 변수로 되어 있다면 
# 계산하고 시각화 할 때 음식 종류를 변수로 지정할 수 있습니다.
# 그래서 컬럼 하나하나를 지정하기 보다는 변수를 넣고 변수의 값 별로 평균을 구해보는게 조금 더 간단합니다.
# 이렇게 변수를 지정하기 위해서는 각 컬럼에 있는 값을 행으로 녹여야 하는데 아래에서 그 방법을 알아보겠습니다.
df.groupby("강수 유형명")["치킨_배달건수"].mean().plot.bar(rot=0)

## 데이터 형태 변환하기

<img src="https://seaborn.pydata.org/_images/data_structure_19_0.png">


### melt로 Tidy data 만들기
pandas의 melt를 사용하면 데이터의 형태를 변경할 수 있습니다. 
배달음식 종류가 컬럼에 있지만 배달음식 종류라는 컬럼을 만들고 변수로 생성해 주면 다른 변수와 비교할 수 있습니다.
melt를 사용해 열에 있는 데이터를 행으로 녹여봅니다.

<img src="https://pandas.pydata.org/docs/_images/reshaping_melt.png" width="500">

* https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
* [Tidy Data 란?](https://vita.had.co.nz/papers/tidy-data.pdf)

In [ ]:
df.head(2)

In [ ]:
# melt 를 사용하게 되면 음식 종류가 행으로 녹아내립니다.
# 그래서 음식을 컬럼별로 하나씩 계산하지 않고 변수로 지정하여 사용할 수 있습니다.
df_melt = df.melt(id_vars=['광역시도명', '시군구명', '날짜', '시간대별 시간', 
                           '강수 유형명', '습도값', '강수량 값', '기온값', 
                           '풍속값', '바람강도 유형명', '동쪽서쪽바람유형명', 
                           '풍향값', '풍향카테고리명'], 
                  var_name="음식", value_name="배달건수")
df_melt.head()

In [ ]:
# metl한 데이터를 사용하면 음식별 배달건수를 변수로 지정해서 시각화 할 수 있습니다.
# 음식 종류마다 평균이나 합계값을 구하는 것보다 비교적 간단하게 구할 수 있습니다.
sns.barplot(data=df_melt, y="음식", x="배달건수", errorbar=None)

In [ ]:
# 그럼 좀 더 자세히 서브플롯을 시각화 하여 시간대별 음식 종류별 배달건수를 알아봅니다.
# 전체적으로 치킨의 배달건수가 가장 많으며 특히 오후와 저녁시간에 많은 것을 볼 수있습니다.
sns.catplot(data=df_melt, x="시간대별 시간", y="배달건수", errorbar=None, kind="bar", 
            col="음식", col_wrap=4)

In [ ]:
# plotly express 로 같은 그래프를 시각화 하고자 하면 histogram 을 사용하면 연산을 하는 그래프를 그릴 수 있습니다.
# plotly 는 동적인 시각화를 지원합니다.
px.histogram(df_melt, x="시간대별 시간", y="배달건수", histfunc="avg", 
             text_auto=True, facet_col="음식", facet_col_wrap=4,
             title="시간대별 평균 음식 배달건수")

## 날씨와 배달건수

In [ ]:
# 평균 배달건수를 비교해 보면 진눈개비, 눈이 올 때 배달이 많다는 것을 확인해 볼 수 있습니다.
# 눈이나 진눈개비가 오는 날은 길도 미끄러운데 배달건수가 많아 배달 안전에 특히 주의가 필요해 보입니다.
sns.barplot(data=df_melt, x="시간대별 시간", y="배달건수", hue="강수 유형명", errorbar=None).set_title("강수 유형별 평균 배달건수")

In [ ]:
# 평균 배달건수는 날씨와 무관하게 치킨이 많으며 진눈개비가 오는 날은 한식, 족발/보쌈 배달도 많습니다.
# 하지만 치킨, 한식, 족발/보쌈 배달은 날씨와 무관하게 다른 음식과 비교하여 배달건수가 많습니다.
sns.catplot(data=df_melt, x="시간대별 시간", y="배달건수", 
            hue="강수 유형명", kind="bar", col="음식", col_wrap=4,
            errorbar=None)

In [ ]:
# plotly express 로 같은 그래프를 시각화 하고자 하면 histogram 을 사용하면 연산을 하는 그래프를 그릴 수 있습니다.
# plotly 는 동적인 시각화를 지원합니다.
# 위에서 그렸던 시각화를 동일하게 그려봅니다. plotly 에서는 seaborn 에서 사용했던 hue 대신 color로 변수 값을 구분하여 표현합니다.
px.histogram(df_melt, x="시간대별 시간", y="배달건수", color="강수 유형명", histfunc="avg", 
             text_auto=True, facet_col="음식", facet_col_wrap=4,
             title="시간대별 평균 음식 배달건수")

* 그럼 처음에 세웠던 가설을 보겠습니다.
* 비가 오는 날에는 기름진 음식이 먹고 싶고, 평소 한식을 많이 먹을 것이라는 가설을 세웠었는데 비가 오는 날 치킨, 족발/보쌈 기름진 음식의 배달이 많으며, 꼭 해당 음식 뿐만 아니라 비가 오는 날 전체적으로 배달건수가 많습니다.
* 날씨가 궂은 날 안전에 유의가 필요한데 배달건수도 많다보니 배달기사분들이 안전하게 배달할 수 있도록 관련된 정책이 만들어졌으면 좋겠습니다.